In [1]:
!pip install pgeocode
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
!pip install folium
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup


print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
import pgeocode
nomi = pgeocode.Nominatim('SE')
nomi.query_postal_code("400 32")
#https://pgeocode.readthedocs.io/en/latest/overview.html#quickstart

postal_code                400 32
country_code                   SE
place_name               Göteborg
state_name        Västra Götaland
state_code                      O
county_name              Göteborg
county_code                  1480
community_name                NaN
community_code                NaN
latitude                  57.7072
longitude                 11.9668
accuracy                        4
Name: 0, dtype: object

In [3]:
#https://www.worldpostalcodes.org/l1/en/se/sweden/map/r2/map-of-postalcodes-in-goteborgs-stad
#https://goteborg.se/wps/wcm/connect/d30328ca-09c0-447d-8f96-954445b47efc/Omr%C3%A5deslista+2021.xlsx?MOD=AJPERES&CONVERT_TO=url&CACHEID=ROOTWORKSPACE-d30328ca-09c0-447d-8f96-954445b47efc-nEuesu6

In [4]:
html_wiki=requests.get('https://www.worldpostalcodes.org/l1/en/se/sweden/map/r2/map-of-postalcodes-in-goteborgs-stad').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_wiki,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <base href="//www.worldpostalcodes.org/"/>
  <title>
   Map of All Postal Codes in Göteborgs Stad July 2021
  </title>
  <meta name="twitter:site" value="@worldpostalcodes"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="" name="twitter:title"/>
  <meta content="" name="twitter:description"/>
  <meta content="//www.worldpostalcodes.org/img/wpc.png" name="twitter:image"/>
  <meta content="" property="og:description"/>
  <meta content="article" property="og:type"/>
  <meta content="Worldpostalcodes.org" property="og:site_name"/>
  <meta content="//www.worldpostalcodes.org/img/wpc.png" property="og:image"/>
  <meta content="" property="og:title"/>
  <meta content="//www.worldpostalcodes.org/" property="og:url"/>
  <meta content="initial-scale=1.0, user-scalable=no" name="viewport"/>
  <script type="app

In [5]:
table_contents=[]
table=soup.find('table')

table2=table.findAll(class_= "mapzoom")

for link in table2:
    cell = {}
    #print(link.get('href'))
    #print(link.get('href').split('(')[1].split(')')[0].split(','))
    cell['PostalCode']=link.get('title').split(' ')[2].replace('-','')
    cell['latitude'] = link.get('href').split('(')[1].split(')')[0].split(',')[0]
    cell['longitude'] = link.get('href').split('(')[1].split(')')[0].split(',')[1]
    table_contents.append(cell)



In [6]:
goteborg_neigh=pd.DataFrame(table_contents)
goteborg_neigh.PostalCode.astype(str)
goteborg_neigh['PostalCode'] = goteborg_neigh['PostalCode'].astype( np.dtype(int))
goteborg_neigh.dtypes
goteborg_neigh.head()
goteborg_neigh.shape
print(goteborg_neigh)
#https://numpy.org/doc/stable/reference/arrays.dtypes.html
#https://stackoverflow.com/questions/39092067/pandas-dataframe-convert-column-type-to-string-or-categorical?rq=1
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html

     PostalCode     latitude    longitude
0         41101  57.70950699  11.98796272
1         41102  57.70892715  11.98471260
2         41103  57.70936203  11.97605896
3         41104  57.71588516  11.98299885
4         41105  57.70920181  11.96586609
5         41106  57.70690536  11.97275734
6         41107  57.70600128  11.97115803
7         41108  57.70523453  11.97196770
8         41109  57.70705414  11.96820831
9         41110  57.70352554  11.96933460
10        41111  57.70828247  11.98045063
11        41112  57.71131134  11.98712444
12        41113  57.70816040  11.96085072
13        41114  57.70595169  11.96469212
14        41115  57.70468903  11.96576118
15        41116  57.70372391  11.96525955
16        41117  57.70277405  11.96435738
17        41118  57.70435333  11.96044159
18        41119  57.70312881  11.95818901
19        41120  57.70166016  11.95948601
20        41121  57.70492554  11.95580101
21        41122  57.69629288  11.95855045
22        41123  57.69689560  11.9

In [7]:
link='https://goteborg.se/wps/wcm/connect/d30328ca-09c0-447d-8f96-954445b47efc/Omr%C3%A5deslista+2021.xlsx?MOD=AJPERES&CONVERT_TO=url&CACHEID=ROOTWORKSPACE-d30328ca-09c0-447d-8f96-954445b47efc-nEuesu6'
stadsdelar = pd.read_excel (link,sheets="alla områden")
stadsdelar['Basområde (NYKO)'] = stadsdelar['Basområde (NYKO)'].astype( np.dtype(int),errors='ignore')
stadsdelar.dtypes
#print(stadsdelar)
stadsdelar.head()
stadsdelar.shape

(1202, 5)

In [8]:
goteborg_neigh=goteborg_neigh.merge(stadsdelar,how='left',left_on='PostalCode',right_on='Basområde (NYKO)')
goteborg_neigh.head()
#print(goteborg_neigh)
#goteborg_neigh.shape

,PostalCode,latitude,longitude,Basområdet aktivt från 2021,Basområde (NYKO),Primärområde,Mellanområde 2021-,Stadsområde
0,41101,57.70950699,11.98796272,Nej,41101.0,413 Skälltorp,71 Backa,SO 4 Hisingen
1,41102,57.70892715,11.98471260,Nej,41102.0,413 Skälltorp,71 Backa,SO 4 Hisingen
2,41103,57.70936203,11.97605896,Nej,41103.0,413 Skälltorp,71 Backa,SO 4 Hisingen
3,41104,57.71588516,11.98299885,Nej,41104.0,412 Backa,71 Backa,SO 4 Hisingen
4,41105,57.70920181,11.96586609,Nej,41105.0,412 Backa,71 Backa,SO 4 Hisingen


In [9]:
goteborg_neigh=goteborg_neigh.drop(['Basområdet aktivt från 2021', 'Basområde (NYKO)'], axis=1)
goteborg_neigh.head()
goteborg_neigh.shape

(482, 6)

In [10]:
address = 'Gothenburg Sweden'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Gothenburg Swenden are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Gothenburg Swenden are 57.7072326, 11.9670171.


In [11]:
# create map of New York using latitude and longitude values
map_gothenburg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(goteborg_neigh['latitude'], goteborg_neigh['longitude'], goteborg_neigh['Stadsområde'], goteborg_neigh['Primärområde']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gothenburg)  
    
map_gothenburg

In [12]:
CLIENT_ID = 'IU4KFW03GFBWDLW1C13ZACV2UAMKMSPG5ITQTLOISQ2WXRTJ' # your Foursquare ID
CLIENT_SECRET = 'GHYAGLAVOXRDGVZN3INEN5FU3EPTUWSZG4ZOSK1A5QKWLECE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IU4KFW03GFBWDLW1C13ZACV2UAMKMSPG5ITQTLOISQ2WXRTJ
CLIENT_SECRET:GHYAGLAVOXRDGVZN3INEN5FU3EPTUWSZG4ZOSK1A5QKWLECE


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
#gothenburg_venues = getNearbyVenues(names=goteborg_neigh['Primärområde'],
                                   #latitudes=goteborg_neigh['latitude'],
                                   #longitudes=goteborg_neigh['longitude']
                                  #)

In [15]:
html_wiki=requests.get('https://osm4wiki.toolforge.org/cgi-bin/wiki/wiki-osm.pl?project=sv&article=Kategori%3AStadsdelar_i_G%25C3%25B6teborg').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_wiki,'lxml')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="OpenStreetMap with Locations from Wikipedia" name="description"/>
  <meta content="Software by User:Plenz @ de.wikipedia" name="generator"/>
  <link href="woicon.ico" rel="shortcut icon"/>
  <title>
   OpenStreetMap for Wikipedia - Wikipedia article: Kategori:Stadsdelar_i_Göteborg
  </title>
  <style type="text/css">
   body       {margin:0px; padding:4px; font-family:'arial'; font-size:0.8em;}
      div        {padding:0px; margin:0px;}
      div.main   {width:100%; height:100%; margin-top:4px;
                  border-style:solid; border-width:1px; border-color:grey;}
      div.momo   {width:0%;  height:0%;  background-image:url(trapa.png); cursor:w-resize;
                  position:absolute; top:0px; left:0px; z-index:995;}


In [16]:
table_contents=[]
table=soup.findAll('table')
print(table)



[<table cellpadding="0px" cellspacing="0px" class="all">
<tr>
<td align="left">
<table cellpadding="0px" cellspacing="0px" class="tool" id="toolbar" width="100%">
<tr valign="top">
<td style="text-align:left;" width="30%">
                 
                <a onclick="openall(-1);" style="cursor:pointer;" title="close all sections">
<img border="0" src="open0.png"/></a>
<a onclick="openall(+1);" style="cursor:pointer;" title="open all sections">
<img border="0" src="open1.png"/></a>
</td>
<td style="text-align:center;" width="30%">
<a onclick="checkall(false);" style="cursor:pointer;" title="hide all icons">
<img border="0" src="check0.png"/></a>
<a onclick="checkall(true);" style="cursor:pointer;" title="show all icons">
<img border="0" src="check1.png"/></a>
</td>
<td style="text-align:center;" width="10%">
<a onclick="opentoolz();" style="cursor:pointer;" title="Settings">
<img border="0" src="tools.png"/></a>
</td>
<td style="text-align:right;" width="30%">
<a onclick="resize(+1);"

In [17]:
table1= soup.find_all('a', href=True, text=True)
#print(table1)
#['a','input']
#print(table1[0])
print(table1[0])
#print(table1[0].get('href').split('/'))
#print(table1[0].get('href').split('/')[4])
type(table1[0].title)

<a href="https://sv.wikipedia.org/wiki/Angered" target="_newiki" title="Source: Wikipedia article">Angered</a>


NoneType

In [18]:
table2= soup.find_all('input')
print(table2[0].get('value').split(','))

['12.0333333333333', '57.8', '0']


In [19]:
table_contentsd=[]

table1= soup.find_all('a', href=True, text=True)
for link in table1:
    if link.text=='Click':
        pass
    else:
        cell = {}
        cell['District']=link.text
        #print(link.get('href').split('/')[4])
        print(link.text)
        table_contentsd.append(cell)


Angered
Annedal, Göteborg
Arendal, Göteborg
Askim
Backa, Göteborg
Bagaregården
Bergsjön
Biskopsgården
Björlanda
Bräcke, Göteborg
Brämaregården
Bö, Göteborg
Delsjön (stadsdel)
Fiskebäck, Göteborg
Färjestaden, Göteborg
Gamlestaden, Göteborg
Guldheden
Gullbergsvass
Gårda
Gårdsten
Haga, Göteborg
Heden, Göteborg
Hjällbo
Högsbo
Inom Vallgraven
Johanneberg
Järnbrott
Kallebäck
Kortedala
Krokslätt
Kungsladugård, Göteborg
Kviberg
Kvillebäcken
Kyrkbyn, Göteborg
Kärra, Göteborg
Kålltorp
Landala
Lilleby
Lindholmen, Göteborg
Lorensberg
Lundbyvassen
Lunden, Göteborg
Majorna
Masthugget
Nordstaden
Nya Varvet
Näset, Göteborg
Olivedal
Olofstorp
Olskroken
Pustervik
Rambergsstaden
Rud (stadsdel)
Rödbo
Rödjan, Göteborg
Sandarna
Sannegården
Skogome
Skår, Göteborg
Slottsskogen
Stampen
Stigberget, Göteborg
Styrsö
Syrhåla
Säve
Sävenäs
Sörred
Tingstadsvassen
Tolered
Torp (stadsdel)
Torslanda, Göteborg
Tuve
Tynnered
Utby
Vasastaden, Göteborg
Älvsborg (stadsdel)
Änggården
Önnered


In [20]:
table_contentsc=[]
table2= soup.find_all('input')
for link in table2:
    
    if link.get('value')==None:
        pass
    else:
        cell = {}
        cell['latitude']=link.get('value').split(',')[1]#Please verify and correct
        cell['longitude']=link.get('value').split(',')[0]
        table_contentsc.append(cell)


In [21]:
#table= soup.find('table')
#print(table)
#table1= table.findAll('a')
#a=[]

#for tag in table1:
 #   cell = {}
  #  txt=tag.text
   # if txt==None:
       # pass
    #elif len(txt.strip()) <= 1:
     #   pass
    #3elif tag.tex=='◄':
     #   pass
    #else:     
        
     #   cell['Neighborhood'] = tag.text
        
    #table_contents.append(cell)


#print(table_contents)       

        
#print(tag.text.split())

#for tag in table:
    #print(table.a)

#print(table1)
#['a','input']
#print(table1[0])
#print(table1[0])
#print(table1[0].get('href').split('/'))
#print(table1[0].get('href').split('/')[4])
#type(table1[0].title)

In [22]:
goteborg_neigh=pd.DataFrame(columns=['District','longitude','latitude'])

goteborg_districts=pd.DataFrame(table_contentsd)
goteborg_coordinates=pd.DataFrame(table_contentsc)
goteborg_neigh['District']=goteborg_districts['District']
goteborg_neigh['longitude']=goteborg_coordinates['longitude']
goteborg_neigh['latitude']=goteborg_coordinates['latitude']
goteborg_neigh

,District,longitude,latitude
0,Angered,12.0333333333333,57.8
1,"Annedal, Göteborg",11.9591333333333,57.6935472222222
2,"Arendal, Göteborg",11.821375,57.7042388888889
3,Askim,11.9313888888889,57.635
4,"Backa, Göteborg",11.982,57.745
5,Bagaregården,12.0091666666667,57.7188888888889
6,Bergsjön,12.0604138888889,57.7592916666667
7,Biskopsgården,11.888,57.722
8,Björlanda,11.825,57.763
9,"Bräcke, Göteborg",11.9001666666667,57.7079166666667


In [23]:
# create map of New York using latitude and longitude values
map_gothenburg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(goteborg_neigh['latitude'], goteborg_neigh['longitude'], goteborg_neigh['District'], goteborg_neigh['District']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gothenburg)  
    
map_gothenburg

In [24]:
goteborg_venues = getNearbyVenues(names=goteborg_neigh['District'],
                                   latitudes=goteborg_neigh['latitude'],
                                   longitudes=goteborg_neigh['longitude']
                                  )

Angered
Annedal, Göteborg
Arendal, Göteborg
Askim
Backa, Göteborg
Bagaregården
Bergsjön
Biskopsgården
Björlanda
Bräcke, Göteborg
Brämaregården
Bö, Göteborg
Delsjön (stadsdel)
Fiskebäck, Göteborg
Färjestaden, Göteborg
Gamlestaden, Göteborg
Guldheden
Gullbergsvass
Gårda
Gårdsten
Haga, Göteborg
Heden, Göteborg
Hjällbo
Högsbo
Inom Vallgraven
Johanneberg
Järnbrott
Kallebäck
Kortedala
Krokslätt
Kungsladugård, Göteborg
Kviberg
Kvillebäcken
Kyrkbyn, Göteborg
Kärra, Göteborg
Kålltorp
Landala
Lilleby
Lindholmen, Göteborg
Lorensberg
Lundbyvassen
Lunden, Göteborg
Majorna
Masthugget
Nordstaden
Nya Varvet
Näset, Göteborg
Olivedal
Olofstorp
Olskroken
Pustervik
Rambergsstaden
Rud (stadsdel)
Rödbo
Rödjan, Göteborg
Sandarna
Sannegården
Skogome
Skår, Göteborg
Slottsskogen
Stampen
Stigberget, Göteborg
Styrsö
Syrhåla
Säve
Sävenäs
Sörred
Tingstadsvassen
Tolered
Torp (stadsdel)
Torslanda, Göteborg
Tuve
Tynnered
Utby
Vasastaden, Göteborg
Älvsborg (stadsdel)
Änggården
Önnered


In [25]:
print(goteborg_venues.shape)
goteborg_venues.head()

(1036, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Angered,57.8,12.0333333333333,Colorama,57.801570,12.033634,Arts & Crafts Store
1,Angered,57.8,12.0333333333333,Gårdstens Centrum,57.803917,12.031979,Shopping Mall
2,Angered,57.8,12.0333333333333,Siazco Enterprises,57.801802,12.026353,Business Service
3,"Annedal, Göteborg",57.6935472222222,11.9591333333333,I Love Pizza,57.694059,11.956864,Pizza Place
4,"Annedal, Göteborg",57.6935472222222,11.9591333333333,Dubbel Dubbel,57.694358,11.955436,Dumpling Restaurant


In [26]:
goteborg_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Angered,3,3,3,3,3,3
"Annedal, Göteborg",47,47,47,47,47,47
"Arendal, Göteborg",1,1,1,1,1,1
Askim,5,5,5,5,5,5
"Backa, Göteborg",3,3,3,3,3,3
Bagaregården,6,6,6,6,6,6
Bergsjön,5,5,5,5,5,5
Biskopsgården,3,3,3,3,3,3
Brämaregården,19,19,19,19,19,19


In [27]:
print('There are {} uniques categories.'.format(len(goteborg_venues['Venue Category'].unique())))

There are 171 uniques categories.


In [28]:
# one hot encoding
goteborg_onehot = pd.get_dummies(goteborg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
goteborg_onehot['Neighborhood'] = goteborg_venues['Neighborhood'] 



# move neighborhood column to the first column
fixed_columns = [goteborg_onehot.columns[-1]] + list(goteborg_onehot.columns[:-1])
goteborg_onehot = goteborg_onehot[fixed_columns]

goteborg_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Champagne Bar,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cruise Ship,Dance Studio,Deli / Bodega,Department Store,Design Studio,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Farm,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Moving Target,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Opera House,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Track,Trail,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Angered,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Angered,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Angered,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Annedal, Göteborg",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Annedal, Göteborg",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
goteborg_onehot.shape
goteborg_grouped = goteborg_onehot.groupby('Neighborhood').mean().reset_index()
goteborg_grouped 

,Neighborhood,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Champagne Bar,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cruise Ship,Dance Studio,Deli / Bodega,Department Store,Design Studio,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Farm,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Moving Target,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Opera House,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Track,Trail,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Angered,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,"Annedal, Göteborg",0.021277,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.021277,0.042553,0.000000,0.000

In [30]:
num_top_venues = 5

for hood in goteborg_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = goteborg_grouped[goteborg_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Angered----
                 venue  freq
0  Arts & Crafts Store  0.33
1     Business Service  0.33
2        Shopping Mall  0.33
3  American Restaurant  0.00
4            Nightclub  0.00


----Annedal, Göteborg----
                venue  freq
0                Café  0.09
1       Grocery Store  0.06
2         Pizza Place  0.06
3  Italian Restaurant  0.06
4          Steakhouse  0.04


----Arendal, Göteborg----
                     venue  freq
0  Scandinavian Restaurant   1.0
1      American Restaurant   0.0
2          Motorcycle Shop   0.0
3            Moving Target   0.0
4                Multiplex   0.0


----Askim----
               venue  freq
0              Diner   0.2
1  Convenience Store   0.2
2           Bus Stop   0.2
3        Pizza Place   0.2
4             Bakery   0.2


----Backa, Göteborg----
                 venue  freq
0             Bus Stop  0.33
1        Grocery Store  0.33
2                 Farm  0.33
3  American Restaurant  0.00
4            Nightclub  0.00


----Baga

                             venue  freq
0                  Motorcycle Shop   0.5
1                     Home Service   0.5
2              American Restaurant   0.0
3                        Nightclub   0.0
4  Molecular Gastronomy Restaurant   0.0


----Sävenäs----
                  venue  freq
0                 Plaza  0.17
1  Fast Food Restaurant  0.17
2               Stadium  0.17
3     Electronics Store  0.17
4       Thai Restaurant  0.17


----Tingstadsvassen----
                             venue  freq
0                           Tunnel   1.0
1              American Restaurant   0.0
2                        Nightclub   0.0
3  Molecular Gastronomy Restaurant   0.0
4                  Motorcycle Shop   0.0


----Tolered----
                        venue  freq
0                    Bus Stop   0.4
1  Construction & Landscaping   0.2
2                 Pizza Place   0.2
3                      Bakery   0.2
4         American Restaurant   0.0


----Torp (stadsdel)----
                  venue 

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = goteborg_grouped['Neighborhood']

for ind in np.arange(goteborg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(goteborg_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angered,Arts & Crafts Store,Shopping Mall,Business Service,Wine Bar,Farm,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market
1,"Annedal, Göteborg",Café,Pizza Place,Grocery Store,Italian Restaurant,Bar,Restaurant,Sushi Restaurant,Steakhouse,Scandinavian Restaurant,Bus Stop
2,"Arendal, Göteborg",Scandinavian Restaurant,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
3,Askim,Pizza Place,Convenience Store,Diner,Bus Stop,Bakery,Farm,Food Stand,Food & Drink Shop,Food,Flea Market
4,"Backa, Göteborg",Bus Stop,Farm,Grocery Store,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market


In [35]:
# set number of clusters
kclusters = 7

goteborg_grouped_clustering = goteborg_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(goteborg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([4, 0, 6, 5, 5, 4, 4, 4, 0, 4], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

goteborg_merged = goteborg_neigh

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
goteborg_merged = goteborg_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='District')

goteborg_merged.head() # check the last columns!

,District,longitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angered,12.0333333333333,57.8,4.0,Arts & Crafts Store,Shopping Mall,Business Service,Wine Bar,Farm,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market
1,"Annedal, Göteborg",11.9591333333333,57.6935472222222,0.0,Café,Pizza Place,Grocery Store,Italian Restaurant,Bar,Restaurant,Sushi Restaurant,Steakhouse,Scandinavian Restaurant,Bus Stop
2,"Arendal, Göteborg",11.821375,57.7042388888889,6.0,Scandinavian Restaurant,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
3,Askim,11.9313888888889,57.635,5.0,Pizza Place,Convenience Store,Diner,Bus Stop,Bakery,Farm,Food Stand,Food & Drink Shop,Food,Flea Market
4,"Backa, Göteborg",11.982,57.745,5.0,Bus Stop,Farm,Grocery Store,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market


In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import math

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(goteborg_merged['latitude'], goteborg_merged['longitude'], goteborg_merged['District'], goteborg_merged['Cluster Labels']):
    
    if (math.isnan(cluster)):
       cluster=0
    
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=2.0).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2c7ef7', '#2adddd', '#80ffb4', '#d4dd80', '#ff7e41', '#ff0000']


In [81]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 0, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Annedal, Göteborg",Café,Pizza Place,Grocery Store,Italian Restaurant,Bar,Restaurant,Sushi Restaurant,Steakhouse,Scandinavian Restaurant,Bus Stop
10,Brämaregården,Café,Food & Drink Shop,Playground,Beer Bar,Pizza Place,Plaza,Coffee Shop,Pool Hall,Pub,Restaurant
12,Delsjön (stadsdel),Campground,Track,Scandinavian Restaurant,Wine Bar,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
15,"Gamlestaden, Göteborg",Brewery,Asian Restaurant,Pub,Gym / Fitness Center,Italian Restaurant,Restaurant,Light Rail Station,Wine Bar,Exhibit,Food & Drink Shop
17,Gullbergsvass,Restaurant,Hotel,Harbor / Marina,Scandinavian Restaurant,Men's Store,Go Kart Track,Cruise Ship,Gym / Fitness Center,Food,Fish Market
18,Gårda,Bus Stop,Light Rail Station,Climbing Gym,Restaurant,Multiplex,Japanese Restaurant,Stadium,Wine Bar,Farm,Food
20,"Haga, Göteborg",Café,Pub,Bar,Coffee Shop,Wine Bar,Plaza,Italian Restaurant,Indie Movie Theater,Indian Restaurant,Burger Joint
21,"Heden, Göteborg",Restaurant,Italian Restaurant,Hotel,Scandinavian Restaurant,Burger Joint,Lounge,Theater,Pub,Park,Nightclub
24,Inom Vallgraven,Hotel,Coffee Shop,Pub,Café,Italian Restaurant,Burger Joint,Scandinavian Restaurant,Plaza,Bookstore,Food Truck
25,Johanneberg,Sushi Restaurant,Light Rail Station,Restaurant,Pizza Place,Scandinavian Restaurant,Hotel,Bus Stop,Café,Lake,Spa


In [82]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 1, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Syrhåla,Trail,Wine Bar,Exhibit,Food Truck,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant


In [83]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 2, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Tingstadsvassen,Tunnel,Wine Bar,Exhibit,Food Truck,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant


In [84]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 3, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Rödbo,Sporting Goods Shop,Wine Bar,French Restaurant,Food Truck,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant


In [85]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 4, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angered,Arts & Crafts Store,Shopping Mall,Business Service,Wine Bar,Farm,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market
5,Bagaregården,Convenience Store,Indian Restaurant,Light Rail Station,Café,Theater,Sushi Restaurant,Design Studio,Diner,Food & Drink Shop,Food
6,Bergsjön,Light Rail Station,Plaza,Music Venue,Pizza Place,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
7,Biskopsgården,Market,Light Rail Station,Lake,Wine Bar,Fast Food Restaurant,Food Truck,Food Stand,Food & Drink Shop,Food,Flea Market
11,"Bö, Göteborg",Light Rail Station,Electronics Store,Pizza Place,Spa,Fast Food Restaurant,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market
16,Guldheden,Shopping Plaza,Bus Station,Restaurant,Grocery Store,Light Rail Station,Bar,Wine Bar,Farm,Food & Drink Shop,Food
19,Gårdsten,Shopping Mall,Flea Market,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Fish Market,Fast Food Restaurant,Farm
22,Hjällbo,Plaza,Pizza Place,Bus Line,Bus Station,Grocery Store,Light Rail Station,Exhibit,Food,Flea Market,Fish Market
23,Högsbo,Grocery Store,Pizza Place,Thai Restaurant,Plaza,Bus Station,Light Rail Station,Supermarket,Convenience Store,Gym / Fitness Center,Tram Station
26,Järnbrott,Clothing Store,Sporting Goods Shop,Bakery,Stadium,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market


In [86]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 5, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Askim,Pizza Place,Convenience Store,Diner,Bus Stop,Bakery,Farm,Food Stand,Food & Drink Shop,Food,Flea Market
4,"Backa, Göteborg",Bus Stop,Farm,Grocery Store,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market
14,"Färjestaden, Göteborg",Playground,Park,Bus Stop,Electronics Store,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
41,"Lunden, Göteborg",Bus Stop,Hotel,Turkish Restaurant,Pizza Place,Arts & Crafts Store,Ice Cream Shop,Café,Grocery Store,Sushi Restaurant,Exhibit
48,Olofstorp,Bus Stop,Pizza Place,Moving Target,Wine Bar,Exhibit,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant
68,Tolered,Bus Stop,Pizza Place,Construction & Landscaping,Bakery,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market
73,Utby,Playground,Rock Climbing Spot,Bus Stop,Baseball Field,Pizza Place,Nature Preserve,Design Studio,Food,Flea Market,Fish Market


In [87]:
goteborg_merged.loc[goteborg_merged['Cluster Labels'] == 6, goteborg_merged.columns[[0] + list(range(4, goteborg_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Arendal, Göteborg",Scandinavian Restaurant,Wine Bar,Exhibit,Food Stand,Food & Drink Shop,Food,Flea Market,Fish Market,Fast Food Restaurant,Farm
